# Template Notebook

In [3]:
import numpy as np
import pandas as pd
import os
from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [4]:
spec = model_spec.get('average_word_vec')
spec.num_words = 30000
spec.seq_len = 16
spec.wordvec_dim = 100

# Find optimal sequence length
text_lengths = data_df['v2'].apply(lambda x: len(x.split()))
print(f"Mean length: {text_lengths.mean()}, Median length: {text_lengths.median()}")



Mean length: 15.494436468054559, Median length: 12.0


In [5]:
# Load data using the DataLoader
data = DataLoader.from_csv(
    filename='../Data/spam_utf8.csv',  # Use the UTF-8 version of the file
    text_column='v2',  
    label_column='v1', 
    model_spec=spec,
    delimiter=',',
    shuffle=True,
    is_training=True
)

# Split the data
train_data, test_data = data.split(0.9)


2024-04-15 09:30:51.362188: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-04-15 09:30:51.362224: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: dewgong
2024-04-15 09:30:51.362230: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: dewgong
2024-04-15 09:30:51.362765: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.67.0
2024-04-15 09:30:51.362785: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.67.0
2024-04-15 09:30:51.362789: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 550.67.0
2024-04-15 09:30:51.363570: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical oper

In [6]:
# Build the model
model = text_classifier.create(train_data, model_spec=spec, epochs=5, 
                               validation_data=test_data)

156/156 [==============================] - 1s 7ms/step - loss: 0.2214 - accuracy: 0.9211 - val_loss: 0.0796 - val_accuracy: 0.9803
Epoch 2/2
156/156 [==============================] - 0s 3ms/step - loss: 0.0604 - accuracy: 0.9800 - val_loss: 0.0540 - val_accuracy: 0.9857
Epoch 3/3
156/156 [==============================] - 0s 2ms/step - loss: 0.0398 - accuracy: 0.9882 - val_loss: 0.0508 - val_accuracy: 0.9910
Epoch 4/4
156/156 [==============================] - 0s 2ms/step - loss: 0.0303 - accuracy: 0.9910 - val_loss: 0.0522 - val_accuracy: 0.9928
Epoch 5/5
156/156 [==============================] - 0s 2ms/step - loss: 0.0244 - accuracy: 0.9932 - val_loss: 0.0560 - val_accuracy: 0.9928


In [7]:
model_dir = '../models'
tflite_filename = 'spam.tflite'
tflite_path = os.path.join(model_dir, tflite_filename)

# Export the model as TensorFlow Lite
model.export(export_dir=model_dir, export_format=ExportFormat.TFLITE)

# Find the exported .tflite file
default_tflite_path = os.path.join(model_dir, 'model.tflite')

# Rename the default 'model.tflite' to 'spam.tflite'
if os.path.exists(tflite_path):
    os.remove(tflite_path)  # Remove if already exists to overwrite
os.rename(default_tflite_path, tflite_path)


2024-04-15 09:30:57.857066: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2024-04-15 09:30:58.144378: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-04-15 09:30:58.144533: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-04-15 09:30:58.145973: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.003ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.

2024-04-15 09:30:58.195424: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2024-04-15 09:30:58.195469: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2024-04-15 09:30:58.214688: 